# Урок 8. Рекуррентные нейронные сети RNN LSTM GRU

## Задание

* построить свёрточные архитектуры
* построить различные архитектуры с RNN
* построить совместные архитектуры CNN -> RNN и (RNN -> CNN)
* сделать выводы что получилось лучше

## Загрузка библиотек

In [15]:
!pip install stop_words pymorphy2

import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking, Flatten
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.callbacks import EarlyStopping

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
url = '/content/drive/MyDrive/Colab Notebooks/data/gb_NLP_les7_data/'

df_train = pd.read_csv(url + 'data/train.csv')
df_test = pd.read_csv(url + 'data/test.csv')
df_val = pd.read_csv(url + 'data/val.csv')

df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [3]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

## Предобработка

In [4]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [5]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

In [6]:
y_train = df_train['class'].values
y_val = df_val['class'].values

## SimpleRNN

In [7]:
model = Sequential()

model.add(
  Embedding(input_dim=word_count,
            input_length=training_length,
            output_dim=30,
            trainable=True,
            mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
  optimizer='adam',
  loss='binary_crossentropy',
  metrics=['accuracy']
)

In [8]:
early_stopping=EarlyStopping(monitor='val_loss')

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 31s 84ms/step - loss: 0.5586 - accuracy: 0.6998 - val_loss: 0.4929 - val_accuracy: 0.7557


In [9]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 0s 7ms/step - loss: 0.5044 - accuracy: 0.7439


Test score: 0.5044305920600891
Test accuracy: 0.7438610196113586


## LSTM

In [10]:
model = Sequential()

model.add(
  Embedding(input_dim=word_count,
            input_length=training_length,
            output_dim=30,
            trainable=True,
            mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
  optimizer='adam',
  loss='binary_crossentropy',
  metrics=['accuracy']
)

early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 76s 229ms/step - loss: 0.5563 - accuracy: 0.7053 - val_loss: 0.4933 - val_accuracy: 0.7552


In [11]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 16ms/step - loss: 0.5174 - accuracy: 0.7393


Test score: 0.5173622369766235
Test accuracy: 0.7393202185630798


## GRU

In [12]:
model = Sequential()

model.add(
  Embedding(input_dim=word_count,
            input_length=training_length,
            output_dim=30,
            trainable=True,
            mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
  optimizer='adam',
  loss='binary_crossentropy',
  metrics=['accuracy']
)

early_stopping=EarlyStopping(monitor='val_loss')

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 67s 201ms/step - loss: 0.5549 - accuracy: 0.7038 - val_loss: 0.4944 - val_accuracy: 0.7534


In [13]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 15ms/step - loss: 0.5128 - accuracy: 0.7408


Test score: 0.5127639174461365
Test accuracy: 0.7407750487327576


## LSTM + CNN

In [17]:
model = Sequential()

model.add(
  Embedding(input_dim=word_count,
            input_length=training_length,
            output_dim=30,
            trainable=True,
            mask_zero=True))
# model.add(Masking(mask_value=0.0))
model.add(LSTM(256, return_sequences="True"))
model.add(Conv1D(128, 3))
model.add(GlobalMaxPool1D())
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

model.compile(
  optimizer='adam',
  loss='binary_crossentropy',
  metrics=['accuracy']
)

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 27, 30)            7743240   
_________________________________________________________________
lstm_3 (LSTM)                (None, 27, 256)           293888    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 25, 128)           98432     
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 128)               0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)               

In [19]:
early_stopping=EarlyStopping(monitor='val_loss')

hh = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
45/45 [==============================] - 1s 23ms/step - loss: 0.5114 - accuracy: 0.7437


Test score: 0.5114206671714783
Test accuracy: 0.743684709072113


## CNN + LSTM

In [24]:
model = Sequential()

model.add(
  Embedding(input_dim=word_count,
            input_length=training_length,
            output_dim=30,
            trainable=True,
            mask_zero=True))
model.add(Conv1D(128, 3))
model.add(LSTM(128, return_sequences="True"))
model.add(Dropout(0.1))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

model.compile(
  optimizer='adam',
  loss='binary_crossentropy',
  metrics=['accuracy']
)

model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 27, 30)            7743240   
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 25, 128)           11648     
_________________________________________________________________
lstm_8 (LSTM)                (None, 25, 128)           131584    
_________________________________________________________________
dropout_7 (Dropout)          (None, 25, 128)           0         
_________________________________________________________________
dense_11 (Dense)             (None, 25, 32)            4128      
_________________________________________________________________
dropout_8 (Dropout)          (None, 25, 32)            0         
_________________________________________________________________
dense_12 (Dense)             (None, 25, 1)           

In [25]:
early_stopping=EarlyStopping(monitor='val_loss')

hh = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
45/45 [==============================] - 1s 12ms/step - loss: 0.6660 - accuracy: 0.5483


Test score: 0.6660130620002747
Test accuracy: 0.5482801198959351


## Вывод

Лучший результат показывает SimpleRNN